In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [2]:
url = "https://raw.githubusercontent.com/ga-students/DS-SF-24/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)

Explore the dataset and decide which variables suffer from missing data

In [3]:
len(OzoneData)

153

In [4]:
OzoneData.describe()

C:\Users\Allison\Anaconda2\lib\site-packages\numpy\lib\function_base.py:3823: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,153.000000,153.000000,153.000000,153.000000
mean,42.129310,185.931507,9.957516,77.882353,6.993464,15.803922
std,32.987885,90.058422,3.523001,9.465270,1.416522,8.864520
min,1.000000,7.000000,1.700000,56.000000,5.000000,1.000000
25%,NaN,NaN,7.400000,72.000000,6.000000,8.000000
50%,NaN,NaN,9.700000,79.000000,7.000000,16.000000
75%,NaN,NaN,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


Answer: It looks like both Ozone and Solar.R are missing values.

Let's drop rows that have missing values in all the columns you identified above. 

Hint: in dropna() if you set how = 'all', it will only drop columns that are suffering from missing values at all variables you introduce in subset. If you want to get rid of the row that contains missing values in any of the variables you specify, then you shall set how = 'any'

df.dropna(how = 'all',subset = ['Var1', 'Var2', 'Var3'],inplace = True)

The above code will check if all 3 variables specified in df have missing values, if they all have missing values it will drop that row. 

In [5]:
OzoneData.dropna(how = 'all',subset = ['Ozone', 'Solar.R'],inplace = True)

In [6]:
len(OzoneData)

151

In [7]:
OzoneData.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,116.000000,146.000000,151.000000,151.000000,151.000000,151.000000
mean,42.129310,185.931507,9.941722,78.165563,7.019868,15.801325
std,32.987885,90.058422,3.524984,9.198138,1.406984,8.832531
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,NaN,NaN,7.400000,73.000000,6.000000,8.000000
50%,NaN,NaN,9.700000,79.000000,7.000000,16.000000
75%,NaN,NaN,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


Spoiler! If everything is going according to plan you shall be left by 151 observations. Also, it seemed like the first two variables had missing values. Now please make a copy of your dataframe into a dataframe named OzoneImputeMean. Also, please use mean of the variables to fill in missing values in OzoneImputeMean.

Hint: copy OzoneData this way:

OzoneImputeMean = OzoneData.copy()

In [8]:
OzoneImputeMean = OzoneData.copy()

OzoneImputeMean['Ozone'].fillna(value = np.mean(OzoneImputeMean['Ozone']), inplace = True)

OzoneImputeMean['Solar.R'].fillna(value = np.mean(OzoneImputeMean['Solar.R']), inplace = True)

In [9]:
#Check if missing values are filled

OzoneImputeMean.describe()

,Ozone,Solar.R,Wind,Temp,Month,Day
count,151.000000,151.000000,151.000000,151.000000,151.000000,151.000000
mean,42.129310,185.931507,9.941722,78.165563,7.019868,15.801325
std,28.884028,88.544727,3.524984,9.198138,1.406984,8.832531
min,1.000000,7.000000,1.700000,57.000000,5.000000,1.000000
25%,21.000000,119.000000,7.400000,73.000000,6.000000,8.000000
50%,42.129310,197.000000,9.700000,79.000000,7.000000,16.000000
75%,46.500000,257.000000,11.500000,85.000000,8.000000,23.000000
max,168.000000,334.000000,20.700000,97.000000,9.000000,31.000000


Now it's time for imputing using linear regression lines

In [10]:
####Before we start let's define dummy variables for Month - don't worry about day!

MonthDummy = pd.get_dummies(OzoneData.Month, prefix = 'Month')
del MonthDummy['Month_9']

OzoneData = pd.concat([OzoneData, MonthDummy], axis = 1)

OzoneData.head()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
0,41.0,190.0,7.4,67,5,1,1.0,0.0,0.0,0.0
1,36.0,118.0,8.0,72,5,2,1.0,0.0,0.0,0.0
2,12.0,149.0,12.6,74,5,3,1.0,0.0,0.0,0.0
3,18.0,313.0,11.5,62,5,4,1.0,0.0,0.0,0.0
5,28.0,NaN,14.9,66,5,6,1.0,0.0,0.0,0.0


In [11]:
#now let's explore correlation matrix

OzoneData.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.601547,0.698360,0.164519,-0.013226,-0.302963,-0.112006,0.277961,0.291808
Solar.R,0.348342,1.000000,-0.056792,0.275840,-0.075301,-0.150275,-0.024601,0.023998,0.176744,-0.076390
Wind,-0.601547,-0.056792,1.000000,-0.466032,-0.175317,0.042365,0.237781,0.046054,-0.144638,-0.166105
Temp,0.698360,0.275840,-0.466032,1.000000,0.397427,-0.136876,-0.637816,0.050753,0.318103,0.321680
Month,0.164519,-0.075301,-0.175317,0.397427,1.000000,-0.007727,-0.702257,-0.362131,-0.007201,0.355246
Day,-0.013226,-0.150275,0.042365,-0.136876,-0.007727,1.000000,0.011003,-0.017044,0.011471,0.011471
Month_5,-0.302963,-0.024601,0.237781,-0.637816,-0.702257,0.011003,1.000000,-0.242766,-0.247805,-0.247805
Month_6,-0.112006,0.023998,0.046054,0.050753,-0.362131,-0.017044,-0.242766,1.000000,-0.253080,-0.253080
Month_7,0.277961,0.176744,-0.144638,0.318103,-0.007201,0.011471,-0.247805,-0.253080,1.000000,-0.258333
Month_8,0.291808,-0.076390,-0.166105,0.321680,0.355246,0.011471,-0.247805,-0.253080,-0.258333,1.000000


Create 3 datasets. Let's name the first one OzoneData_NoMissing which contains no missing values. DataSet2 is called OzoneData_SolarMissingDrop which does not have any Solar.R missing value. DataSet 3 is named OzoneData_OzoneMissingDrop which does not have any Ozone Missing Value.

Hint: df2 = df1.copy() only copies df1 without changing anything in df1 for instance, here we create OzoneData_NoMissing

OzoneData_NoMissing = OzoneData.dropna(how = 'any', subset = ['Ozone', 'Solar.R'], inplace = False).copy()

In [12]:
OzoneData_NoMissing = OzoneData.dropna(how = 'any', subset = ['Ozone', 'Solar.R'], inplace = False).copy()

OzoneData_NoMissing.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.612497,0.698541,0.142885,-0.005190,-0.284989,-0.113476,0.284103,0.276269
Solar.R,0.348342,1.000000,-0.127183,0.294088,-0.074067,-0.057754,-0.015976,-0.001897,0.192732,-0.066002
Wind,-0.612497,-0.127183,1.000000,-0.497190,-0.194496,0.049871,0.232019,0.187716,-0.221211,-0.155720
Temp,0.698541,0.294088,-0.497190,1.000000,0.403972,-0.096546,-0.627510,0.013446,0.355139,0.318096
Month,0.142885,-0.074067,-0.194496,0.403972,1.000000,-0.009001,-0.793583,-0.246301,-0.081527,0.273183
Day,-0.005190,-0.057754,0.049871,-0.096546,-0.009001,1.000000,0.008325,-0.055263,0.018174,0.072426
Month_5,-0.284989,-0.015976,0.232019,-0.627510,-0.793583,0.008325,1.000000,-0.156015,-0.290485,-0.268515
Month_6,-0.113476,-0.001897,0.187716,0.013446,-0.246301,-0.055263,-0.156015,1.000000,-0.164285,-0.151860
Month_7,0.284103,0.192732,-0.221211,0.355139,-0.081527,0.018174,-0.290485,-0.164285,1.000000,-0.282748
Month_8,0.276269,-0.066002,-0.155720,0.318096,0.273183,0.072426,-0.268515,-0.151860,-0.282748,1.000000


In [13]:
OzoneData_SolarMissingDrop = OzoneData.dropna(how = 'any', subset = ['Solar.R'], inplace = False).copy()

OzoneData_SolarMissingDrop.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.612497,0.698541,0.142885,-0.005190,-0.284989,-0.113476,0.284103,0.276269
Solar.R,0.348342,1.000000,-0.056792,0.275840,-0.075301,-0.150275,-0.024601,0.023998,0.176744,-0.076390
Wind,-0.612497,-0.056792,1.000000,-0.453847,-0.162253,0.025307,0.232622,0.038284,-0.157609,-0.131326
Temp,0.698541,0.275840,-0.453847,1.000000,0.380970,-0.131936,-0.633554,0.054409,0.326818,0.298509
Month,0.142885,-0.075301,-0.162253,0.380970,1.000000,-0.006977,-0.690119,-0.373376,-0.010165,0.338571
Day,-0.005190,-0.150275,0.025307,-0.131936,-0.006977,1.000000,0.023492,-0.036164,-0.007303,0.058650
Month_5,-0.284989,-0.024601,0.232622,-0.633554,-0.690119,0.023492,1.000000,-0.242237,-0.247309,-0.232031
Month_6,-0.113476,0.023998,0.038284,0.054409,-0.373376,-0.036164,-0.242237,1.000000,-0.264036,-0.247725
Month_7,0.284103,0.176744,-0.157609,0.326818,-0.010165,-0.007303,-0.247309,-0.264036,1.000000,-0.252912
Month_8,0.276269,-0.076390,-0.131326,0.298509,0.338571,0.058650,-0.232031,-0.247725,-0.252912,1.000000


In [14]:
OzoneData_OzoneMissingDrop = OzoneData.dropna(how = 'any', subset = ['Ozone'], inplace = False).copy()

OzoneData_OzoneMissingDrop.corr()

,Ozone,Solar.R,Wind,Temp,Month,Day,Month_5,Month_6,Month_7,Month_8
Ozone,1.000000,0.348342,-0.601547,0.698360,0.164519,-0.013226,-0.302963,-0.112006,0.277961,0.291808
Solar.R,0.348342,1.000000,-0.127183,0.294088,-0.074067,-0.057754,-0.015976,-0.001897,0.192732,-0.066002
Wind,-0.601547,-0.127183,1.000000,-0.511075,-0.206250,0.069262,0.240945,0.188684,-0.202192,-0.195804
Temp,0.698360,0.294088,-0.511075,1.000000,0.421166,-0.105814,-0.633969,0.010795,0.342250,0.346628
Month,0.164519,-0.074067,-0.206250,0.421166,1.000000,-0.004238,-0.804128,-0.236518,-0.072529,0.293270
Day,-0.013226,-0.057754,0.069262,-0.105814,-0.004238,1.000000,-0.002127,-0.039985,0.042956,0.014482
Month_5,-0.302963,-0.015976,0.240945,-0.633969,-0.804128,-0.002127,1.000000,-0.155882,-0.288889,-0.288889
Month_6,-0.112006,-0.001897,0.188684,0.010795,-0.236518,-0.039985,-0.155882,1.000000,-0.155882,-0.155882
Month_7,0.277961,0.192732,-0.202192,0.342250,-0.072529,0.042956,-0.288889,-0.155882,1.000000,-0.288889
Month_8,0.291808,-0.066002,-0.195804,0.346628,0.293270,0.014482,-0.288889,-0.155882,-0.288889,1.000000


What seems to be the list of best variables to define Ozone? How about Solar.R?

Answer: I'm not entirely sure what to say here having not yet used the regression model, so I tried correlation matrices for each of the three datasets. From the correlation matrix in each dataset, it looks like wind is strongly negatively correlated to Ozone, and Temp is strongly positively correlated to Ozone. Correlations for Solar.R are not nearly as strong, but one could say that both Ozone and Temp are somewhat positively correlated to Solar.R.

Now let's use a regression model to predict Ozone. First drop NaN values in Ozone and save it in OzoneDroppedValues_Ozone. Then run a regression line on variables of interest and check significancy of your model if in a multi class dummy variable case, you see only a few of the dummy variables are not significant but the majority are, you shall either drop all or keep all. Otherwise, selecting the base dummy will become important use these variables ['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8'] to predict.

Hint: For this part you need to use

X = OzoneData_NoMissing[['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8']]

y = OzoneData_NoMissing['Ozone']

In [15]:
#I"m not sure why we need to make a new copy of the dataset called 
#OzoneDroppedValues_Ozone... the example (so my code) uses OzoneData_NoMissing.
#What am I missing?

OzoneDroppedValues_Ozone = OzoneData.dropna(how = 'any', subset = ['Ozone'], inplace = True)

X1 = OzoneData_NoMissing[['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']]

y1 = OzoneData_NoMissing['Ozone']

lm1 = smf.ols(formula = 'y1 ~ X1', data = OzoneData_NoMissing).fit()
print lm1.pvalues

#trying to zip these p values with the column names
list_of_all_variables = ['Solar.R','Wind','Temp','Month_5','Month_6','Month_7','Month_8']

for a, b in zip(lm1.pvalues, list_of_all_variables):
    print a,b


Intercept    2.318427e-03
X1[0]        2.957179e-02
X1[1]        7.777659e-06
X1[2]        2.740750e-07
X1[3]        1.823435e-02
X1[4]        8.812922e-01
X1[5]        2.289481e-01
X1[6]        5.824286e-02
dtype: float64
0.00231842666027 Solar.R
0.0295717866234 Wind
7.77765897028e-06 Temp
2.74074954156e-07 Month_5
0.0182343513296 Month_6
0.881292152303 Month_7
0.22894811788 Month_8


In [16]:
#now fill in null values of OzoneData['Ozone'] by predicted values
#Here you can see the guideline to fill in missing values of Ozone.

linreg = LinearRegression()

linreg.fit(X1,y1)

X = OzoneData[['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8']]

OzoneData['Ozone_predict'] = linreg.predict(X)
OzoneData['Ozone'].fillna(value = OzoneData['Ozone_predict'], inplace = True)



#Pay attention to how we make predictions


OzoneData.head(10)

#This doesn't fill in my missing data... what am I doing wrong? 

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#now repeat previous steps for Solar.R variable using ['Ozone', 'Wind', 'Temp']

X2 = OzoneData[['Ozone', 'Wind','Temp']]

y2 = OzoneData['Solar.R']

lm2 = smf.ols(formula = 'y2 ~ X2', data = OzoneData).fit()
print lm2.pvalues

#check P-values of your model - if a variable is not significant, drop it

In [ ]:
#now fill in null values of OzoneData['Solar.R'] by predicted values

linreg = LinearRegression()

linreg.fit(X2,y2)

X = OzoneData[['Ozone', 'Wind']] #'Temp' doesn't appear to be significant.

OzoneData['Solar.R_predicted'] = linreg.predict(X)
OzoneData['Solar.R'].fillna(value = OzoneData['Solar.R_predicted'], inplace = True)

OzoneData.head(10)

Now check your filled data - if your predicted values are more than maximum or less than minimum, replace them by max and min.

In [ ]:
X2 = OzoneData[['Ozone', 'Wind','Temp']]

y2 = OzoneData['Solar.R']

y2_min = y2.min()
y2_max = y2.max()

OzoneData.loc[(OzoneData['Solar.R_predicted'] < y2_min), 'Solar.R_predicted'] = y2_min 
OzoneData.loc[(OzoneData['Solar.R_predicted'] > y2_max), 'SOlar.R_predicted'] = y2_max

print('Minimum Values after adjustment %f:'  %OzoneData['Solar.R_predicted'].min())


OzoneData['Solar.R'].fillna(value = OzoneData['Solar.R_predicted'], inplace = True)


OzoneData.head(20)


Now check your dataset and see if imputed values are in acceptable range. If there is anything wrong with it just point it out. Fix "Out of range" values if you find any.

Answer: 

Bonus: Repeat the above procedure, this time fill in missing values using regression with errors.

In [ ]:
url = "https://raw.githubusercontent.com/ga-students/DS-SF-24/master/Data/ozone.csv"
OzoneData = pd.read_csv(url)

OzoneData_DroppedValues = OzoneData.dropna(subset = ['Ozone'])

linreg = LinearRegression()

X = OzoneData_DroppedValues[['Solar.R', 'Wind', 'Temp', 'Month_5', 'Month_6', 'Month_7', 'Month_8']]
y = OzoneData_DroppedValues['Ozone']

linreg.fit(X,y)

y_hat = linreg.predict(X)

Standard_error = ( sum( (y-y_hat)**2)/(len(y)-2))**.5

print(Standard_error)

OzoneData['Ozone_predict'] = linreg.predict(X)
OzoneData['Ozone'].fillna(value = (OzoneData['Ozone_predict']+Standard_error), inplace = True)



In [ ]:
OzoneData.head(10)